In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow.keras import Sequential, layers
from keras.layers import Dense, InputLayer, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.callbacks import EarlyStopping as ES
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-02-29 09:18:24.447969: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 09:18:24.448083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 09:18:24.735918: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_dir = '/kaggle/input/splitted-10-classes-of-food-plates/food_data_splitted/food_data_splitted/train'
test_dir= '/kaggle/input/splitted-10-classes-of-food-plates/food_data_splitted/food_data_splitted/test'

In [3]:
train_gen = ImageDataGenerator(
    rescale=1./255.,
    validation_split=0.2,
)
test_gen = ImageDataGenerator(rescale=1./255.)

In [4]:
batchSize = 32
imageShape = (224, 224)

In [5]:
train_ds = train_gen.flow_from_directory(train_dir,target_size=imageShape,batch_size=batchSize, subset='training', class_mode='categorical')
val_ds = train_gen.flow_from_directory(train_dir,target_size=imageShape,batch_size=batchSize, subset='validation', class_mode='categorical')
test_ds = test_gen.flow_from_directory(test_dir,target_size=imageShape,batch_size=batchSize, class_mode='categorical')

Found 6400 images belonging to 10 classes.
Found 1600 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [6]:
def get_model():
    
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    for layers in base_model.layers:
        layers.trainable = False
        
    base_model_output = base_model.output
    
    x = Flatten()(base_model_output)
    x = Dense(512, activation='relu')(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=x)
    return model


In [7]:
model = get_model()
model.compile(loss='categorical_crossentropy', # No need to OHE target
              optimizer='adam',
              metrics=['accuracy'])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
history = model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=50,
                        steps_per_epoch=len(train_ds),
                    validation_steps=len(val_ds),
                    callbacks=[ES(monitor="val_loss", patience =5)])

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  2/200 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.1328 - loss: 14.0100

I0000 00:00:1709198381.700401     129 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1709198381.732465     129 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - accuracy: 0.5889 - loss: 6.9463

W0000 00:00:1709198429.805903     129 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


200/200 ━━━━━━━━━━━━━━━━━━━━ 76s 295ms/step - accuracy: 0.5898 - loss: 6.9035 - val_accuracy: 0.7319 - val_loss: 0.8716
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 199us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/50


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


200/200 ━━━━━━━━━━━━━━━━━━━━ 30s 144ms/step - accuracy: 0.9238 - loss: 0.2304 - val_accuracy: 0.7550 - val_loss: 1.0658
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 107us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 30s 143ms/step - accuracy: 0.9805 - loss: 0.0639 - val_accuracy: 0.7931 - val_loss: 0.9270
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 89us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 29s 141ms/step - accuracy: 0.9966 - loss: 0.0145 - val_accuracy: 0.7944 - val_loss: 0.9293


In [9]:
model.evaluate(test_ds)

63/63 ━━━━━━━━━━━━━━━━━━━━ 19s 307ms/step - accuracy: 0.7915 - loss: 1.0403


W0000 00:00:1709198617.341747     130 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


[0.997878909111023, 0.7950000166893005]

In [44]:
import numpy as np
from tensorflow.keras.preprocessing import image
img_path = '/kaggle/input/testtt/test.jpg'
img = image.load_img(img_path, target_size=imageShape)

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

# Make predictions
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


In [45]:
predictions , predicted_class_index

(array([[3.9702441e-09, 4.9961471e-07, 6.0218497e-09, 9.9999654e-01,
         3.0004912e-06, 2.5297939e-10, 5.4229146e-09, 5.9536035e-08,
         2.9653927e-08, 2.3244038e-09]], dtype=float32),
 3)

In [46]:
class_labels = ['fish_and_chips', 'french_toast', 'fried_calamari', 'garlic_bread',
                'grilled_salmon', 'hamburger', 'ice_cream', 'lasagna', 'macaroni_and_cheese','macarons']

# Map the predicted class index to the corresponding class label
predicted_label = class_labels[predicted_class_index]

print("Predicted class label:", predicted_label)

Predicted class label: garlic_bread


In [23]:
!pip install gTTS

In [47]:
from gtts import gTTS
import IPython.display as ipd

# Define a function to convert text to audio and play it
def text_to_audio(text, output_file='prediction_audio.mp3'):
    tts = gTTS(text=text, lang='en')
    tts.save(output_file)
    return output_file


# Define a dictionary to map class indices to class labels
class_mapping = {
    0: 'fish and chips',
    1: 'french toast',
    2: 'fried calamari',
    3: 'garlic bread',
    4: 'grilled salmon',
    5: 'hamburger',
    6: 'ice cream',
    7: 'lasagna',
    8: 'macaroni and cheese',
    9: 'macarons'
}

# Predicted class index
predicted_class_index = predicted_class_index  # Replace with the actual predicted class index

# Get the predicted class label from the mapping
predicted_label = class_mapping[predicted_class_index]

# Convert the predicted class label to audio and play it
audio_file = text_to_audio(predicted_label)
ipd.Audio(audio_file)

In [49]:
model.save('/kaggle/working/model.h5')